In [1]:
import torch
import torch.nn as nn
import importlib

In [2]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
def fit_pca(img, pca):
    return pca.fit_transform(img)

def dbscan_clustering(min_samples=5):
    return DBSCAN(min_samples=min_samples)

def normalization(data):
    scaler = StandardScaler()
    return scaler.fit_transform(data)

def tsne_visualization(n_components=2, perplexity=30.0, learning_rate='auto', verbose=1):
    return TSNE(n_components=n_components, perplexity=perplexity, learning_rate=learning_rate, verbose=verbose)

def clustering_result(kmeans, n_categorys):
    image_cluster_dict = {}
    image_cluster_categorys = {}
    for i, m in enumerate(kmeans):
        image_cluster_dict[f'{m}'] = 0
        image_cluster_categorys[f'{m}'] = np.zeros(2)       
    for i, m in enumerate(kmeans):
        image_cluster_dict[f'{m}'] += 1
        image_cluster_categorys[f'{m}'][n_categorys[i]] += 1        
    for i in image_cluster_dict:
        image_cluster_dict[f'{i}'] = round(image_cluster_dict[f'{i}']/len(n_categorys), 2)
    return image_cluster_dict, image_cluster_categorys

In [3]:
mod = importlib.import_module('torchvision.models')
model = mod.resnet18(pretrained=True)
model.fc = nn.Identity()

In [4]:
def normalize(data):
    min_value = torch.min(data)
    max_value = torch.max(data)
    return (data-min_value)/(max_value-min_value)

In [5]:
import os
path = 'D:/kaggle-Fruit-Dataset'
folder_lists = [os.path.join(path, folder) for folder in os.listdir(path)]
image_lists = []
for folder in folder_lists:
    image_lists.extend([os.path.join(folder, name) for name in os.listdir(folder)])

In [6]:
from PIL import Image
import numpy as np

data_lists = []
for image_path in image_lists:
    image = Image.open(image_path).convert('RGB')
    image = np.array(image)
    data_lists.append(image)


In [7]:
print(len(data_lists))

2607


In [8]:
batch_size = 16
input = []
result = []
for idx, data in enumerate(data_lists):
    tmp = torch.FloatTensor(data)
    tmp = normalize(tmp)
    input.append(tmp.permute(2, 0, 1))
    if len(input)==batch_size or idx==len(data_lists)-1:
        input = torch.stack(input, dim=0)
        result.extend(model(input).detach().cpu().numpy())
        input = []

In [9]:
result = np.stack(result, axis=0)
print(result.shape)

(2607, 512)


In [10]:
pca = PCA(n_components=100)
pca_result = fit_pca(result, pca)
norm_result = normalization(pca_result)

In [11]:
clustering = kmeans_clustering(number_clusters=6, random_state=1)
clustering_result = clustering(norm_result)

TypeError: 'KMeans' object is not callable

In [ ]:
pca_result.shape